Assignment 4 - Classifying handwritten digits using Numpy and Keras.
--
For this assignment, you should complete the exercises in this notebook. It is similar to the notebook posted for binary logistic regression. 

Look for requests containing the text **"your code"**. E.g. "put your code here", "replace None by your code", etc.
If there is no such request in a cell, just run the cell.

In [2]:
%tensorflow_version 2.x
import tensorflow as tf
from tensorflow import keras
print(tf.__version__)

import numpy as np
import matplotlib.pyplot as plt
import time

UsageError: Line magic function `%tensorflow_version` not found.


In [3]:
# Let's load the dataset of handwritten digits. 

(X, Y), (X_test, Y_test) = keras.datasets.fashion_mnist.load_data()

class_names = ['T-shirt/top', 'Trouser', 'Pullover', 'Dress', 'Coat',
               'Sandal', 'Shirt', 'Sneaker', 'Bag', 'Ankle boot']

plt.figure(figsize=(10,10))
for i in range(25):
    plt.subplot(5,5,i+1)
    plt.xticks([])
    plt.yticks([])
    plt.grid(False)
    plt.imshow(X[i], cmap=plt.cm.binary)
    plt.xlabel(class_names[Y[i]])
plt.show()

NameError: name 'keras' is not defined

In [0]:
# We will reshape (flatten) X arrays so that they become rank 2 arrays. 
# We will reshape the Y arrays so that they are not rank 1 arrays but rank 2 arrays. 
# They should be rank 2 arrays.

X = X.reshape(X.shape[0], X.shape[1]*X.shape[2])
X_test = X_test.reshape(X_test.shape[0], X_test.shape[1]*X_test.shape[2])

Y = Y.reshape((Y.shape[0],1))
Y_test = Y_test.reshape((Y_test.shape[0],1))

print("Train dataset shape", X.shape, Y.shape)
print("Test dataset shape", X_test.shape, Y_test.shape)

print("Y =", Y)

m   = X.shape[0] 
n_x = X.shape[1]

C = 10

Train dataset shape (60000, 784) (60000, 1)
Test dataset shape (10000, 784) (10000, 1)
Y = [[9]
 [0]
 [0]
 ...
 [3]
 [0]
 [5]]


Exercise 1 - One Hot Encoding
--

Convert Y to "one-hot" encoding. E.g. if the original Y is 
$$
Y = \begin{bmatrix}
    1 \\
    5 \\
    9  
\end{bmatrix}
$$
the new Y should be
$$
Y = \begin{bmatrix}
    0 & 1 & 0 & 0 & 0 & 0 & 0 & 0 & 0 & 0\\
    0 & 0 & 0 & 0 & 0 & 1 & 0 & 0 & 0 & 0\\
    0 & 0 & 0 & 0 & 0 & 0 & 0 & 0 & 0 & 1\\ 
\end{bmatrix}
$$

In [4]:
# Toward this goal, let's check what np.arange(C) produces
np.arange(C)

NameError: name 'np' is not defined

In [0]:
# Let's see again what Y is
Y

array([[9],
       [0],
       [0],
       ...,
       [3],
       [0],
       [5]], dtype=uint8)

In [0]:
# What would broadcasting these arrays together would look like? 
# Let's check.

a,b = np.broadcast_arrays(np.arange(C), Y)

print("broadcasted np.arange(C) = \n", a)
print("broadcasted Y = \n", b)

broadcasted np.arange(C) = 
 [[0 1 2 ... 7 8 9]
 [0 1 2 ... 7 8 9]
 [0 1 2 ... 7 8 9]
 ...
 [0 1 2 ... 7 8 9]
 [0 1 2 ... 7 8 9]
 [0 1 2 ... 7 8 9]]
broadcasted Y = 
 [[9 9 9 ... 9 9 9]
 [0 0 0 ... 0 0 0]
 [0 0 0 ... 0 0 0]
 ...
 [3 3 3 ... 3 3 3]
 [0 0 0 ... 0 0 0]
 [5 5 5 ... 5 5 5]]


In [1]:
# If we compare np.arange(C) with Y using the equality sign ==, 
# the numpy broadcasting will do its magic to give us what we want. 
# Try it out. Then assign the result to a new variable Y_new. 
# Don't worry for the "True" and "False" values looking like strings. 
# They behave in fact like numbers, i.e. True is 1, False is 0.
# Finally, assign Y_new to Y so that we have to deal with Y in rest of the notebook.
# Cast the boolean values of Y_new to integer by calling Y_new.astype(int)

# Put your code in place of None objects 

Y_new = np.arange(C) == Y
print("Y_new=", Y_new)
Y =Y_new.astype(int)
print("Y=",Y)

Y_new_test =  np.arange(C) == Y_test
print("Y_new_test=", Y_new_test)
Y_test = Y_new_test.astype(int)
print("Y_test=",Y_test)

NameError: name 'np' is not defined

*Expected output*

<pre>
Y_new= [[False False False ... False False  True]
 [ True False False ... False False False]
 [ True False False ... False False False]
 ...
 [False False False ... False False False]
 [ True False False ... False False False]
 [False False False ... False False False]]
Y= [[0 0 0 ... 0 0 1]
 [1 0 0 ... 0 0 0]
 [1 0 0 ... 0 0 0]
 ...
 [0 0 0 ... 0 0 0]
 [1 0 0 ... 0 0 0]
 [0 0 0 ... 0 0 0]]
Y_new_test= [[False False False ... False False  True]
 [False False  True ... False False False]
 [False  True False ... False False False]
 ...
 [False False False ... False  True False]
 [False  True False ... False False False]
 [False False False ... False False False]]
Y_test= [[0 0 0 ... 0 0 1]
 [0 0 1 ... 0 0 0]
 [0 1 0 ... 0 0 0]
 ...
 [0 0 0 ... 0 1 0]
 [0 1 0 ... 0 0 0]
 [0 0 0 ... 0 0 0]]
</pre>

Exercise 2 - The Softmax Function
--
(Adapted from Andrew Ng's exercise in Coursera, deeplearning.ai)

Implement a softmax function using numpy. Softmax is a normalizing function used when the algorithm needs to classify two or more classes. 

**Instructions**:
- for $x \in \mathbb{R}^{1\times n}$ 
$$\mbox{softmax}(x) = \mbox{softmax}\left(\begin{bmatrix}
    x_1  &&
    x_2 &&
    ...  &&
    x_n  
\end{bmatrix}\right) = \begin{bmatrix}
     \frac{e^{x_1}}{\sum_{j}e^{x_j}}  &&
    \frac{e^{x_2}}{\sum_{j}e^{x_j}}  &&
    ...  &&
    \frac{e^{x_n}}{\sum_{j}e^{x_j}} 
\end{bmatrix} $$ 

- for a matrix $x \in \mathbb{R}^{m \times n}$  
$$\mbox{softmax}(x) = \mbox{softmax}\begin{bmatrix}
    x_{11} & x_{12} & x_{13} & \dots  & x_{1n} \\
    x_{21} & x_{22} & x_{23} & \dots  & x_{2n} \\
    \vdots & \vdots & \vdots & \ddots & \vdots \\
    x_{m1} & x_{m2} & x_{m3} & \dots  & x_{mn}
\end{bmatrix} = \begin{bmatrix}
    \frac{e^{x_{11}}}{\sum_{j}e^{x_{1j}}} & \frac{e^{x_{12}}}{\sum_{j}e^{x_{1j}}} & \frac{e^{x_{13}}}{\sum_{j}e^{x_{1j}}} & \dots  & \frac{e^{x_{1n}}}{\sum_{j}e^{x_{1j}}} \\
    \frac{e^{x_{21}}}{\sum_{j}e^{x_{2j}}} & \frac{e^{x_{22}}}{\sum_{j}e^{x_{2j}}} & \frac{e^{x_{23}}}{\sum_{j}e^{x_{2j}}} & \dots  & \frac{e^{x_{2n}}}{\sum_{j}e^{x_{2j}}} \\
    \vdots & \vdots & \vdots & \ddots & \vdots \\
    \frac{e^{x_{m1}}}{\sum_{j}e^{x_{mj}}} & \frac{e^{x_{m2}}}{\sum_{j}e^{x_{mj}}} & \frac{e^{x_{m3}}}{\sum_{j}e^{x_{mj}}} & \dots  & \frac{e^{x_{mn}}}{\sum_{j}e^{x_{mj}}}
\end{bmatrix} = \begin{pmatrix}
    \mbox{softmax}\text{(first row of x)}  \\
    \mbox{softmax}\text{(second row of x)} \\
    ...  \\
    \mbox{softmax}\text{(last row of x)} \\
\end{pmatrix} $$

In [0]:
def softmax(x):
    # This normalization is done to avoid number overflow. 
    # See: https://stats.stackexchange.com/questions/304758/softmax-overflow
    x_normalized = x - np.max(x,axis=1,keepdims=True)

    # Create an array x_exp by applying np.exp() element-wise to x_normalized. 
    # Put your code here (one line)

    # Create an array x_sum that contains the sum of each row of x_exp. 
    # Use np.sum(..., axis = 1, keepdims = True).
    # Put your code here (one line)
    
    # Compute softmax(x) by dividing x_exp by x_sum. 
    # It should automatically use numpy broadcasting.
    # Return this array.
    # Add a very small constant, 1e-15, to each matrix entry (using broadcasting) 
    # in order to avoid any pure-zero entries.
    # This number doesn't make predictions much off, and it solves the log(0) issue
    # in the cross-entropy function.
    # Replace None with your code
    return None


# Let's test
x = np.array([
    [1, 2, 3, 1, 2],
    [9, 5, 1, 0 ,0]])
print("softmax(x) = " + str(softmax(x)))

softmax(x) = None


*Expected output*

<pre>
softmax(x) = [[  6.74508059e-02   1.83350300e-01   4.98397788e-01   6.74508059e-02
    1.83350300e-01]
 [  9.81452586e-01   1.79759312e-02   3.29240664e-04   1.21120871e-04
    1.21120871e-04]]
</pre>

Exercise 3 - Compute one semi-vectorized iteration for softmax
--
Perform one semi-vectorized iteration of gradient descent for the softmax classification.

In [0]:
# First do this for only one training example (the first one, i=0).
# Print out everything you compute, e.g. print("z", z), print("a", a), etc. 

J = 0
w = np.zeros((n_x,C))
b = np.zeros((1,C))

dw = np.zeros((n_x,C));
db = np.zeros((1,C));

i = 0;


# Replace None objects by your code

x = X[i:i+1, : ]  #x is [1,784]
y = None
print("y", y)

z = None
print("z", z)
a = None
print("a", a)
J = None
print("J", J)

dz = None
print("dz", dz)
dw = None
print("dw", dw)
db = None
print("db", db)

y None
z None
a None
J None
dz None
dw None
db None


*Expected output*

<pre>
y [[0 0 0 0 0 0 0 0 0 1]]
z [[0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]]
a [[0.1 0.1 0.1 0.1 0.1 0.1 0.1 0.1 0.1 0.1]]
J 2.3025850929940357
dz [[ 0.1  0.1  0.1  0.1  0.1  0.1  0.1  0.1  0.1 -0.9]]
dw [[0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 ...
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]]
db [[ 0.1  0.1  0.1  0.1  0.1  0.1  0.1  0.1  0.1 -0.9]]
</pre>

In [0]:
#one iteration, semi-vectorized

J = 0
w = np.zeros((n_x,C))
b = np.zeros((1,C))

dw = np.zeros((n_x,C));
db = np.zeros((1,C));

alpha = 0.000001

start_time = time.time()

for i in range(m):
    # Put your code here
    None

J /= m; dw /= m; db /= m

w -= alpha*dw
b -= alpha*db

print("J", J)
print("Time needed: ", time.time() - start_time)

J 0.0
Time needed:  0.004528522491455078


*Expected output*

<pre>
J =  2.302585092995416
Time needed:  5.705857038497925
</pre>

Of course, your running time will be different.

Exercise 4 - Compute one fully-vectorized iteration for softmax
--
Perform one fully-vectorized iteration of gradient descent for the softmax classification.

In [0]:
#one iteration, fully vectorized, no for loop

J = 0
w = np.zeros((n_x,C))
b = np.zeros((1,C))

alpha = 0.000001

start_time = time.time()


#Replace the None objects and alpha*0 by your code.

# Convention: 
# Use capital letter when the variable is for the whole dataset of m train examples.

# X is (55000,784), Y is (55000,10), w is (784,10), b is (1,10)  
Z = None               # Z is  (55000, 10)
A = None              # A is  (55000, 10)
J = None 

dZ = None                 # dZ is (55000, 10)


dw = None                #dw is (784, 10) 
db = None

w -= alpha*0
b -= alpha*0

print("J = ", J)
print("Time needed: ", time.time() - start_time)

J =  None
Time needed:  0.0005636215209960938


*Expected output*

<pre>
J =  2.3025850929940366
Time needed:  0.6775269508361816
</pre>

We observe that the time of the fully vectorized version is almost one order of magnitude smaller. 

Exercise 5 - Compute several fully-vectorized iterations for softmax
--
Perform 100 fully-vectorized iterations of gradient descent for the softmax classification.
Start with doing 10 iterations first, check the accuracy you achieve, then try for 100 iterations. 
Print out the cost after each iteration.

In [0]:
J = 0
w = np.zeros((n_x,C))
b = np.zeros((1,C))

alpha = 0.000001

#Replace the None objects and alpha*0 by your code.

# Convention: 
# Use capital letter when the variable is for the whole dataset of m train examples.

for iter in range(10):
    # X is (55000,784), Y is (55000,10), w is (784,10), b is (1,10)  
    Z = None               # Z is  (55000, 10)
    A = None              # A is  (55000, 10)
    J = None 
    print(iter, J)
    
    dZ = None                 # dZ is (55000, 10)


    dw = None                #dw is (784, 10) 
    db = None

    w -= alpha*0
    b -= alpha*0

0 None
1 None
2 None
3 None
4 None
5 None
6 None
7 None
8 None
9 None


*Expected output*

<pre>
0 2.3025850929940366
1 2.146205548481119
2 2.030819399467471
3 1.9311838198434277
4 1.8436117384575559
5 1.7662245987459277
6 1.6975659726233228
7 1.636412794191129
8 1.581725819043199
9 1.5326221800639164
</pre>

Exercise 6 - Compute the accuracy
--
Compute the accuracy of softmax classification on the train and test datasets.

Use np.argmax(A, 1) and np.argmax(Y, 1) to find the predicted and real class for each example. They return an array of numbers each, e.g. [7 3 9 ..., 8 0 8] and [7 3 4 ..., 5 6 8]. Compare them using ==. You will get an array of booleans, e.g. [ True  True False ..., False False  True]. Sum up the latter using np.sum(). True values will be considered 1, False values will be considered 0, so the sum will tell us how many True values we got. Then divide by Y.shape[0] and multiply by 100 to get the accuracy in percentage.    

In [0]:
# Replace None by your code

def accuracy(A, Y):
    return None

Z = X @ w + b
A = softmax(Z)

Z_test = X_test @ w + b
A_test = softmax(Z_test)

print("Accuracy on the train set is ", accuracy(A,Y))
print("Accuracy on the test set is ", accuracy(A_test,Y_test))

Accuracy on the train set is  None
Accuracy on the test set is  None


*Expected output*

<pre>
Accuracy on the train set is  66.35166666666667
Accuracy on the test set is  65.38
</pre>

Remark. These numbers are for 10 iterations. When you perform 100 iterations, the numbers will be much better.

Exercise 7 - Implement the Softmax classifier using Keras
--
Implement the Softmax classifier using Keras. This is similar to examples shown in class. Use categorical crossentropy for loss function. Use stochastic optimization (rmsprop or adam) with 5 epochs. Produce the accuracy on the test set in the end. 

Exercise 8 - Implement a neural network with one hidden layer using Keras.
--
Turn the previous exercise into a 1-hidden layer neural network with rectified linear units and 15 hidden nodes. The output layer should continue to be softmax.

The hidden layer should be fully connected to the input layer and to the output layer.

Check the accuracy on the test set. If it is lower than 80%, increase the number of hidden nodes until you reach or exceed this level accuracy.